In [1]:
import requests
import urllib
import json 

Onderstaande code is de basis connectie met het API, er kunnen parameters toegevoegd worden om meer gespecificeerde informatie te verkrijgen. Op deze link staan alle parameters die we toe zouden kunnen voegen:
https://developer.yummly.com/documentation

In [2]:
response = requests.get("http://api.yummly.com/v1/api/recipes?_app_id=2b082bfe&_app_key=905dfe0267500a98d67737f446479c11")
response.json()

{u'attribution': {u'html': u"Recipe search powered by <a href='http://www.yummly.co/recipes'><img alt='Yummly' src='https://static.yummly.co/api-logo.png'/></a>",
  u'logo': u'https://static.yummly.co/api-logo.png',
  u'text': u'Recipe search powered by Yummly',
  u'url': u'http://www.yummly.co/recipes/'},
 u'criteria': {u'allowedIngredient': None,
  u'excludedIngredient': None,
  u'q': None},
 u'facetCounts': {},
 u'matches': [{u'attributes': {u'course': [u'Main Dishes']},
   u'flavors': None,
   u'id': u'Honey-Mustard-Slow-Cooker-Chicken-2221106',
   u'imageUrlsBySize': {u'90': u'https://lh3.googleusercontent.com/So235b6of0RRRppQWcWUKWLggtyYzEv2D1d42pv9bc2pQsnqsTsXxfXOhnUKMugp4qN1E12tkrVZwRdSmyYVVw=s90-c'},
   u'ingredients': [u'PERDUE\xae FRESH CUTS\u2122 Chicken Breast Strips',
    u'honey',
    u'stone ground mustard',
    u'orange juice',
    u'soy sauce',
    u'corn starch',
    u'cold water',
    u'couscous'],
   u'rating': 3,
   u'recipeName': u'Honey Mustard Slow Cooker Chick

In [3]:
mediterranean_main_recipes_15min = requests.get("http://api.yummly.com/v1/api/recipes?_app_id=2b082bfe&_app_key=905dfe0267500a98d67737f446479c11&allowedCuisine[]=cuisine^cuisine-mediterranean&allowedCuisine[]=cuisine^cuisine-italian&allowedCuisine[]=cuisine^cuisine-spanish&maxTotalTimeInSeconds=600&allowedCourse[]=course^course-Main Dishes").json()
# converted to json to be sure that the right json format is being used.  
# Recipes which are either itlian, spanish or meditarean, which are a main course and take less than 15 minutes to cook. 

In [4]:
print mediterranean_main_recipes_15min['matches']
# dit was ook even om te testen hoe de file eruit ziet 

[{u'flavors': {u'piquant': 0.0, u'sour': 0.5, u'salty': 0.6666666666666666, u'sweet': 0.16666666666666666, u'bitter': 0.3333333333333333, u'meaty': 0.8333333333333334}, u'rating': 5, u'totalTimeInSeconds': 300, u'ingredients': [u'jumbo shrimp', u'olive oil', u'butter', u'minced garlic', u'lemon juice', u'lemon zest', u'sea salt'], u'smallImageUrls': [u'http://lh4.ggpht.com/iM63E6R_vx-DTpY3U-cfz32hZTFjtoj-9W85tn2t0JQSduqjjF6wi6fi7nJO5Y1j0Ip9g11wfewzJ-8XKimr6A=s90'], u'sourceDisplayName': u"Kalyn's Kitchen", u'recipeName': u'Easy Garlic and Lemon Shrimp', u'attributes': {u'course': [u'Main Dishes'], u'cuisine': [u'Barbecue', u'Italian', u'Asian', u'American']}, u'id': u'Easy-garlic-and-lemon-shrimp-309257', u'imageUrlsBySize': {u'90': u'http://lh6.ggpht.com/jzVeUvT-8y_V_BflWMk3WRmDO8XLm51-N6Ze69jy2uk_yetgzwJJacZUC4m6rhO96IJak0-tWWvZCafBA80sTQ=s90-c'}}, {u'flavors': {u'piquant': 0.0, u'sour': 0.3333333333333333, u'salty': 0.16666666666666666, u'sweet': 0.16666666666666666, u'bitter': 0.5,

In [5]:
from rdflib import Graph, Namespace, URIRef, Literal, XSD, RDF
# create an RDF graph
g = Graph()

REC = Namespace("https://graph.yummly.com/")

g.bind('rec', REC)

In [6]:
for recipe in mediterranean_main_recipes_15min['matches']:
    for ingredient in recipe['ingredients']:
        rec_URI = REC["ingredient-{}".format(ingredient)]
print rec_URI

# dit is even een testje om te kijken of dat met die link is gelukt. Het is gelukt alleen is dit alleen op het eerste ingredient. 

https://graph.yummly.com/ingredient-parsley


In [7]:
print mediterranean_main_recipes_15min['matches'][0].keys()
# dit slaat eigenlijk nergens op maar dit is om te kijken hoe de file is opgebouwd 

[u'flavors', u'rating', u'totalTimeInSeconds', u'ingredients', u'smallImageUrls', u'sourceDisplayName', u'recipeName', u'attributes', u'id', u'imageUrlsBySize']


In [68]:
ingredient_list = []
for recipe in mediterranean_main_recipes_15min['matches']:
    for ingredient in recipe['ingredients']:
        ingredient_list.append(ingredient)
        
recipe_name_list = []
for recipe in mediterranean_main_recipes_15min['matches']:
    recipe_name_list.append(recipe['recipeName'])

cooking_time_list = []
for recipe in mediterranean_main_recipes_15min['matches']:
    cooking_time_list.append(recipe['totalTimeInSeconds'])
    
# dit zijn lijsten met ingredienten, recipes en cookingtime. Zodat hierover geloopt kan worden.

In [82]:
for ingredient in ingredient_list:
    ingredient2 = REC["ingredient-{}".format(ingredient)]
    g.add((ingredient2, RDF.type, Literal(ingredient)))

print ingredient2
# dit is de url die er uit komt. Een beetje vage namen trouwens, maar had niet zo veel inspiratie. Ik had dat ingredient2 gedaan omdat het anders een beetje misliep met twee ingredients....

https://graph.yummly.com/ingredient-parsley


In [77]:
print g.serialize(format='turtle') # Er komt dus hier een error. Ik denk zelf dat het komt door de spaties in de 
# ingredient namen. Ik heb echt heel veel geprobeerd, er allerlei packages maar hij pakt hem gewoon niet... Beetje irritant..
# misschien ziet een van jullie het licht morgen ochtend. Ik heb al urllib.parse gebrobeerd maar dat lukte niet, en ik heb
# ook nog een paar andere packages van urllib geprobeerd maar dat lukte ook niet. Ook had ik nog .replace geprobeerd maar dat
# ging ook niet. Dus misschien heeft iemand anders nog inspiratie. 

Exception: "https://graph.yummly.com/ingredient-Italian bread" does not look like a valid URI, I cannot serialize this as N3/Turtle. Perhaps you wanted to urlencode it?